In [1]:
import deviantart
import requests
import random
import sqlite3
from bs4 import BeautifulSoup
import json
import time
import requests
import requests.auth
import datetime
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import threading
import os
import pandas as pd

In [2]:
def get_token():
         client = BackendApplicationClient(client_id=client_id)
         oauth = OAuth2Session(client=client)
         token = oauth.fetch_token(token_url=TOKEN_URL, client_id=client_id, client_secret=client_secret)
         # Extract the access token
         access_token = token['access_token']
         return access_token

In [3]:
# Example usage:
client_id = "42096"
client_secret = "97080792c6d30a4178965e41f1ca15de"
TOKEN_URL = "https://www.deviantart.com/oauth2/token"
REDIRECT_URI = "https://www.deviantart.com/oauth2/authorize"

In [4]:
def get_response_rate(response):    
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    elif response.status_code == 404:
        return 'user_done'
    elif response.status_code == 429:
        return 'too_many_requests'
    elif response.status_code == 500:
        return 'server error'
    elif response.status_code == 401:
        return 'get new token'

In [5]:
def get_friends(username, page):
    token=get_token()
    api_url=f"https://www.deviantart.com/api/v1/oauth2/user/friends/{username}?access_token={token}"
    response = requests.get(api_url, params={'offset': page, 'limit': 50})
    return response

In [6]:
def get_watchers(username, page):
    token=get_token()
    api_url=f"https://www.deviantart.com/api/v1/oauth2/user/watchers/{username}?access_token={token}"
    response = requests.get(api_url, params={'offset': page, 'limit': 50})
    return response

In [45]:
get_watchers('wolfysilver',1).json()

{'has_more': True,
 'next_offset': 51,
 'results': [{'user': {'userid': 'E54E7980-7DCD-6915-B079-025A343E068D',
    'username': 'mermaidbanana',
    'usericon': 'https://a.deviantart.net/avatars/m/e/mermaidbanana.jpg?12',
    'type': 'premium'},
   'is_watching': False,
   'lastvisit': '2024-11-18T15:58:01-0800',
   'watch': {'friend': True,
    'deviations': True,
    'journals': True,
    'forum_threads': True,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'userid': 'B18E08FB-5B86-D389-91D4-B6D355EE7E14',
    'username': 'NerdyRatlover',
    'usericon': 'https://a.deviantart.net/avatars/n/e/nerdyratlover.jpg?2',
    'type': 'regular'},
   'is_watching': False,
   'lastvisit': '2024-10-26T18:17:00-0700',
   'watch': {'friend': True,
    'deviations': True,
    'journals': True,
    'forum_threads': True,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'userid': '9075EE

In [32]:
def parse_friends(friends):
    users=pd.Dat
    #print(friends.keys())
    #has_more=friends.get('has_more')
    #next_offset=friends['next_offset']
    for i in friends['results']:
        a={ 'username': i['user']['username'],
            'is_watching': i['is_watching'],
            'watches_you': i['watches_you'],
            'last_visit': i['lastvisit'],
            'activity': i['watch']['activity'],
            'collections': i['watch']['collections'],
            'critiques':  i['watch']['critiques'],
            'deviations': i['watch']['deviations'],
            'forum_threads': i['watch']['forum_threads'],
            'friend': i['watch']['friend'],
            'journals': i['watch']['journals'],
            'scraps': i['watch']['scraps']}
        users=users.append(pd.DataFrame.from_dict(a, orient='index').T)
    return (has_more, a)

In [32]:
def parse_watchers(friends):
    users=pd.DataFrame()
    #print(friends.keys())
    #has_more=friends.get('has_more')
    #next_offset=friends['next_offset']
    for i in friends['results']:
        a={ 'username': i['user']['username'],
            'is_watching': i['is_watching'],
            'watches_you': i['watches_you'],
            'last_visit': i['lastvisit'],
            'activity': i['watch']['activity'],
            'collections': i['watch']['collections'],
            'critiques':  i['watch']['critiques'],
            'deviations': i['watch']['deviations'],
            'forum_threads': i['watch']['forum_threads'],
            'friend': i['watch']['friend'],
            'journals': i['watch']['journals'],
            'scraps': i['watch']['scraps']}
        users=users.append(pd.DataFrame.from_dict(a, orient='index').T)
    return (has_more, users)

In [46]:
def get_all_friends(user):
    token=get_token()
    all_friends=pd.DataFrame()
    has_more=True
    for i in range(0,500):
        if has_more==True:
            resp=get_friends(user, i)
            friends=get_response_rate(resp)
            #print(friends)
            if friends=='too_many_requests':
                time.sleep(60)
                friends=get_response_rate(resp)
            if friends != None:
                has_more, users=parse_friends(friends)
            else:
                return 'no friends'
            if len(users)>0:
                all_friends=all_friends.append(users)
            else:
                return 'no friends'
    all_friends.to_csv(path+user+'_friends.csv')
    print(all_friend.info())
    return 

In [50]:
user_friends = get_friends('nestery',1)
friends = get_response_rate(user_friends)

In [36]:
def get_all_watchers(user):
    token=get_token()
    all_friends=pd.DataFrame()
    has_more=True
    for i in range(0,500):
        if has_more==True:
            resp=get_watchers(user, i)
            friends=get_response_rate(resp)
            #print(friends)
            if friends=='too_many_requests':
                time.sleep(60)
                friends=get_response_rate(resp)
            if friends != None:
                has_more, users=parse_friends(friends)
            else:
                return 'no watchers'
            if len(users)>0:
                all_friends=all_friends.append(users)
            else:
                return 'no watchers'
    all_friends.to_csv(path+user+'_watchers.csv')
    print('watchers printed')
    return 

In [ ]:
def get_watchers_and_watching(username):
        """Gets the watchers and watching of a deviant."""
        try:
          # Get watchers
            # Initialize an empty list to store watchers
            watchers_list = []
            
            # Get the initial batch of watchers
            for i in range(0, 500):
                watchers = get_watchers(username,i)
            
            # Check if there are more watchers to fetch
            has_more = watchers['has_more']
            
            while has_more:
                # Get the next batch of watchers using the next_offset
                watchers = self.da.get_watchers(username, offset=watchers['next_offset'])
            
                # Add the usernames to the list
                for watcher in watchers['results']:
                    watchers_list.append(watcher['user'])
            
                # Update has_more for the next iteration
                has_more = watchers['has_more']
            
        
    
            # Get the initial batch of watching users
            has_more, watching = get_friends(username)  # Use get_friends() to get watching list
            
            # Add the usernames to the list
            for watched in watching['results']:
                watching_list.append(watched['user'])
            
            # Check if there are more users to fetch
            has_more = watching['has_more']
            
            while has_more:
                # Get the next batch of users using the next_offset
                watching = self.da.get_friends(username, offset=watching['next_offset'])
            
                # Add the usernames to the list
                for watched in watching['results']:
                    watching_list.append(watched['user'])
            
                # Update has_more for the next iteration
                has_more = watching['has_more']

        except Exception as e:
            if "HTTP Error 429" in str(e):  # Check for HTTP 429 (Too Many Requests)
                print(f"Rate limit hit! Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait for 60 seconds
                return self.get_watchers_and_watching(username, sample_size)  # Retry the request
            elif "HTTP Error 401" in str(e):  # Check for HTTP 429 (Too Many Requests)
                print(f"API returning unauthorised error. Refreshing the token")
                self.refresh_token()
                time.sleep(300)  # Wait for 300 seconds
                return self.get_watchers_and_watching(username, sample_size)  # Retry the request
            else:
                print(f"Error fetching watchers/watching for {username}: {e}")
                return [], []
        except Exception as e:
            print(f"Error fetching watchers/watching for {username}: {e}")
            return [], []

        return watchers_list, watching_list

In [7]:
 def get_user_profile(username):
        token= get_token()
        #url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
        url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error getting profile info: {e}")
            return None
        return response

def parse_user_profile(user):
    s= get_user_profile(user)
    d=json.loads(s.text)
    if 'error' not in d.keys():
        a={'user':user,
        'real_name': d.get('real_name'),
        'profil_url': d['profile_url'],
        'tags': d['tagline'],
        'country': d['countryid'],
        'bio': d['bio'],
        'level': d['artist_level'],
        'specialty' :d['artist_specialty']}
        a.update(d['stats'])
        return a
        print(f'Gather profile information for the {user}')
    else:
        return 'error'

In [15]:
parse_user_profile('flightlesswing1111')

{'username': 'flightlesswing1111',
 'real_name': '',
 'profil_url': 'https://www.deviantart.com/flightlesswing1111',
 'Artist or Not': False,
 'tags': 'cou',
 'country': 'Japan',
 'level': None,
 'specialty': None,
 'stats': {'user_deviations': 447,
  'user_favourites': 1397,
  'user_comments': 120,
  'profile_pageviews': 1207,
  'profile_comments': 45}}

In [ ]:
# Read usernames from CSV
usernames_df = pd.read_csv("/mnt/hdd/maittewa/gathered_usernames1.csv")  # Replace with your CSV file

# Keep track of already scraped usernames
scraped_usernames = set()
if os.path.exists("/mnt/hdd/maittewa/deviants60K_profileAPI.csv"):
    existing_df = pd.read_csv("/mnt/hdd/maittewa/deviants60K_profileAPI.csv")
    scraped_usernames.update(existing_df['username'].tolist())

# Initialize an empty list to store scraped data
all_about_data = []

batch_size = 10
for i in range(0, len(usernames_df), batch_size):
    batch_usernames = usernames_df['username'][i : i + batch_size].tolist()

    for username in batch_usernames:
        if username not in scraped_usernames:
          try:
            api_result = parse_user_profile(username)
                #watchers, watching = scraper.get_watchers_and_watching(username)
            if api_result:
                all_about_data.append(api_result)
            else:
                print(f"Skipping {username} due to scraping error")
                continue
            print(f"Successfully processed user: {username}")  # Print here
              
            time.sleep(random.uniform(5, 15))  # Sleep after each user
          except Exception as e:
                print(f"Error processing user {username}: {e}")
                # Consider logging the error or taking other actions
                time.sleep(random.uniform(5, 15))  # Sleep to avoid potential rate limits


    # Create a DataFrame from the current batch
    batch_df = pd.DataFrame(all_about_data)

    # Save the batch DataFrame to CSV (append if file exists)
    batch_df.to_csv("/mnt/hdd/maittewa/deviants60K_profileAPI.csv", mode="a", header=not os.path.exists("/mnt/hdd/maittewa/deviants60K_profileAPI.csv"), index=False)

    # Clear the list for the next batch
    all_about_data = []

    print(f"Batch {i // batch_size + 1} completed and saved.")
    time.sleep(5)  # Sleep to avoid potential rate limits

print("DeviantArt Api Calling Completed.")

Batch 1 completed and saved.
Batch 2 completed and saved.
Error getting profile info: 400 Client Error: Bad Request for url: https://www.deviantart.com/api/v1/oauth2/user/profile/PedroNeves2024onDA?access_token=9dc6d592faabb5d3694757efa75e6217837d51a30b6d6ca781
Error processing user PedroNeves2024onDA: 'NoneType' object has no attribute 'text'
Batch 3 completed and saved.
Batch 4 completed and saved.
Batch 5 completed and saved.
Batch 6 completed and saved.
Batch 7 completed and saved.
Batch 8 completed and saved.
Batch 9 completed and saved.
Batch 10 completed and saved.
Batch 11 completed and saved.
Batch 12 completed and saved.
Batch 13 completed and saved.
Batch 14 completed and saved.
Batch 15 completed and saved.
Batch 16 completed and saved.
Batch 17 completed and saved.
Batch 18 completed and saved.
Batch 19 completed and saved.
Batch 20 completed and saved.
Batch 21 completed and saved.
Batch 22 completed and saved.
Batch 23 completed and saved.
Error getting profile info: 400

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Successfully processed user: KyleeFournier2008
Batch 3586 completed and saved.
Successfully processed user: anthonypg549
Successfully processed user: CoopaFreeShaVacaDoo
Successfully processed user: SoulessFreak2
Successfully processed user: mekhiedwards
Successfully processed user: CricketSmellyFeet
Successfully processed user: matiriani1999
Successfully processed user: Raging-Lunatic
Successfully processed user: Marcosbatman
Successfully processed user: LoogerWoui
Successfully processed user: nokiabrick
Batch 3587 completed and saved.
Successfully processed user: GloverLand
Successfully processed user: TehRCCarToyStoryGuy
Successfully processed user: ricardobalthazar
Successfully processed user: jayden1282
Successfully processed user: HoekDrip90
Successfully processed user: Taikoshrimp
Successfully processed user: erick2k21
Successfully processed user: Tagirovo2004
Successfully processed user: Cole2009x
Successfully processed user: kalellanderson
Batch 3588 completed and saved.
Succe

In [4]:
!pip install webdriver-manager

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# Update ChromeOptions to specify headless mode
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [27]:
usernames = ["wolfysilver", "nestery", "linked-memories"]

user_groups = set()

for username in usernames:
    user_url = f"https://www.deviantart.com/{username}/about"
    driver.get(user_url)

    section = driver.find_element(By.CSS_SELECTOR, "#group_list_members > div._2u8NU")
    links = section.find_elements(By.TAG_NAME, "a")
    for link in links:
        link_text = link.text.strip()
        last_word = link_text.split()[-1] if link_text else ""
        user_groups.update(last_word)
        

    while True:
        try:
            button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#group_list_members > button"))
            )
            # Scroll to button and then wait before clicking
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()  
            time.sleep(10)  
            button.click()

            # Wait for new content to load (adjust the wait time as needed)
            #WebDriverWait(driver, 20).until(EC.staleness_of(section))

            # Re-locate the section and extract from new links
            section = driver.find_element(By.CSS_SELECTOR, "#group_list_members > div._2u8NU")
            links = section.find_elements(By.TAG_NAME, "a")
            for link in links:
                # Same extraction logic as before
                link_text = link.text.strip()
                last_word = link_text.split()[-1] if link_text else ""
                user_groups.add(last_word)
            
                

        except TimeoutException as e:
            print(f"Error scraping: {e}")
            


            

Error scraping: Message: 
Stacktrace:
#0 0x58be167786ca <unknown>
#1 0x58be16449600 <unknown>
#2 0x58be16498bcb <unknown>
#3 0x58be16498eb1 <unknown>
#4 0x58be164dcb24 <unknown>
#5 0x58be164bb8cd <unknown>
#6 0x58be164da04a <unknown>
#7 0x58be164bb643 <unknown>
#8 0x58be1648bd31 <unknown>
#9 0x58be1648c79e <unknown>
#10 0x58be1674025b <unknown>
#11 0x58be167441f2 <unknown>
#12 0x58be1672d615 <unknown>
#13 0x58be16744d82 <unknown>
#14 0x58be1671225f <unknown>
#15 0x58be16767e68 <unknown>
#16 0x58be16768040 <unknown>
#17 0x58be1677749c <unknown>
#18 0x796ad4a94ac3 <unknown>

Error scraping: Message: 
Stacktrace:
#0 0x58be167786ca <unknown>
#1 0x58be16449600 <unknown>
#2 0x58be16498bcb <unknown>
#3 0x58be16498eb1 <unknown>
#4 0x58be164dcb24 <unknown>
#5 0x58be164bb8cd <unknown>
#6 0x58be164da04a <unknown>
#7 0x58be164bb643 <unknown>
#8 0x58be1648bd31 <unknown>
#9 0x58be1648c79e <unknown>
#10 0x58be1674025b <unknown>
#11 0x58be167441f2 <unknown>
#12 0x58be1672d615 <unknown>
#13 0x58be16744

KeyboardInterrupt: 